# Vertex AI Workbench

install/upgrade Google Cloud services SDK

In [ ]:
!pip3 install --upgrade --quiet google-cloud-aiplatform

In [ ]:
!gcloud config list

In [ ]:
# Enter the Google Cloud project ID, bucket URI, and region
PROJECT_ID = ''
BUCKET_URI = 'gs://'
REGION = ''

In [ ]:
from google.cloud import aiplatform as ai
# aiplatform will serve as an SDK to interact with Vertex AI features
ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

# Loading data from a Google Storage bucket

In [ ]:
import pandas as pd
gcs_path = f'{BUCKET_URI}/dry_bean.csv'

beans = pd.read_csv(gcs_path)

beans.head()

# Exploring data with pandas in Vertex AI


In [ ]:
beans.info()

In [ ]:
feature_names = beans.columns.tolist()

In [ ]:
import seaborn as sns

sns.pairplot(
   beans,
   vars=["Area", "MajorAxisLength", "MinorAxisLength", "Eccentricity", "roundness"],
   hue="Class",
);

In [ ]:
import matplotlib.pyplot as plt

correlation = beans.corr(numeric_only=True)

# Create a square heatmap with center at 0
sns.heatmap(correlation, center=0, square=True, cmap="coolwarm", vmin=-1, vmax=1)

plt.show()



# Running an AutoML training job

In [ ]:
ds = ai.TabularDataset.create(
   display_name="dry_bean_dataset", gcs_source=gcs_path
)

In [ ]:
job = ai.AutoMLTabularTrainingJob(
   display_name="dry-bean-classification",
   optimization_prediction_type="classification",
)

In [ ]:
model = job.run(
   dataset=ds,
   target_column="Class",
   training_fraction_split=0.8,
   validation_fraction_split=0.1,
   test_fraction_split=0.1,
   model_display_name="baseline-classification-model",
   disable_early_stopping=False,
)

# Deploying an AutoML model and generating predictions


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

In [ ]:
prediction = endpoint.predict(
   [
       {
           "Area": "30099",
           "Perimeter": "638.8209999999999",
           "MajorAxisLength": "237.14191130827916",
           "MinorAxisLength": "162.3034300714102",
           "AspectRation": "1.4611022774068396",
           "Eccentricity": "0.7290928631259719",
           "ConvexArea": "30477",
           "EquivDiameter": "195.76321681302556",
           "Extent": "0.8036043251902283",
           "Solidity": "0.9875972044492568",
           "roundness": "0.9268374259664279",
           "Compactness": "0.8255108332939839",
           "ShapeFactor1": "0.007878730566074592",
           "ShapeFactor2": "0.002256976927384019",
           "ShapeFactor3": "0.6814681358857279",
           "ShapeFactor4": "0.9956946453228307",
       }
   ]
)

print(prediction)